In [ ]:
#import sys
#sys.path.append('..')
import deeplabcut
import tensorflow as tf
print(tf.__version__)
import glob, os


# Base path is the path containing the videos divided under subject directories. The path to any video looks like:
# base_path / sub_ID / vid_name.ext (ext can be either avi or mp4 depending on the video)
base_path = '/media/ssd_storage/work_repos/MGH/bootstrap_round3_2019_08_14_videos/'

# Working directory is the path to a folder where all DeepLabCut project files will be stored for the
# current run of experiments.
work_dir = '/media/ssd_storage/work_repos/MGH/deeplabcut_mgh_pose/experiments/'

In [ ]:
# List videos from all subject directories and store them separately as we want different model per subject
videos_bw46 = glob.glob(os.path.join(base_path, 'BW46', '*.avi'))
videos_mg51b = glob.glob(os.path.join(base_path, 'MG51b', '*.avi'))
videos_mg117 = glob.glob(os.path.join(base_path, 'MG117', '*.avi'))
videos_mg118 = glob.glob(os.path.join(base_path, 'MG118', '*.avi'))
videos_mg120b = glob.glob(os.path.join(base_path, 'MG120b', '*.avi'))

# Create new project for each different subject with arguments: project name, experimenter name, video path list,
# working directory path and a flag indicating whether the videos should be copied to project folder from source
# directory or not.
# The return values from these create_new_project function calls will be the path to their respective
# config.yaml files. For example, for BW46 here, the path should be:
# work_dir / 'mgh_pose_dlc_BW46_2-Kalpit-YYYY-MM-DD' / config.yaml
config_path_bw46 = deeplabcut.create_new_project(
    project='mgh_pose_dlc_BW46_2',
    experimenter='Kalpit',
    videos=videos_bw46,
    working_directory=work_dir,
    copy_videos=True
)
config_path_mg51b = deeplabcut.create_new_project(
    project='mgh_pose_dlc_MG51b_2',
    experimenter='Kalpit',
    videos=videos_mg51b,
    working_directory=work_dir,
    copy_videos=True
)
config_path_mg117 = deeplabcut.create_new_project(
    project='mgh_pose_dlc_MG117_2',
    experimenter='Kalpit',
    videos=videos_mg117,
    working_directory=work_dir,
    copy_videos=True
)
config_path_mg118 = deeplabcut.create_new_project(
    project='mgh_pose_dlc_MG118_2',
    experimenter='Kalpit',
    videos=videos_mg118,
    working_directory=work_dir,
    copy_videos=True
)
config_path_mg120b = deeplabcut.create_new_project(
    project='mgh_pose_dlc_MG120b_2',
    experimenter='Kalpit',
    videos=videos_mg120b,
    working_directory=work_dir,
    copy_videos=True
)

In [ ]:
# Extract frames from the videos mentioned in the YAML at config_path and check if they are labeled
# (I generated labels by hand as we had already annotated them. This process is difficult to explain
# and I won't cover it here). For any new project for pose estimation, the annotation tool provided in DLC
# should be used as it makes things streamlined and much easier.
config_path_bw46 = os.path.join(work_dir, 'mgh_pose_dlc_BW46_2-Kalpit-2019-08-14', 'config.yaml')
config_path_mg51b = os.path.join(work_dir, 'mgh_pose_dlc_MG51b_2-Kalpit-2019-08-14', 'config.yaml')
config_path_mg117 = os.path.join(work_dir, 'mgh_pose_dlc_MG117_2-Kalpit-2019-08-14', 'config.yaml')
config_path_mg118 = os.path.join(work_dir, 'mgh_pose_dlc_MG118_2-Kalpit-2019-08-14', 'config.yaml')
config_path_mg120b = os.path.join(work_dir, 'mgh_pose_dlc_MG120b_2-Kalpit-2019-08-14', 'config.yaml')

# Set the config_path to a value according to which subject you'd want to process
config_path = config_path_mg120b
# extract_frames function extracts frames from the videos using uniform or k-means sampling (to make sure
# there is enough variability in pose frames to be annotated). These extracted frames should then be annotated
# using their annotation GUI. I had not done that (I imported annotations we had already done), but it should be
# easy to follow from their documentation on Github.
# deeplabcut.extract_frames(config_path, 'automatic', 'uniform', crop=False)

# Once the labeling is done, call this function to check the labels by plotting them on the frames and saving
# these frames with annotations plotted. Check these frames to validate the annotation accuracy.
# deeplabcut.check_labels(config_path)

In [ ]:
# Run this function to create the training dataset for different subjects. This function divides the annotations
# and the frames into training and validation sets that are used to train / validate the pose estimation model.
deeplabcut.create_training_dataset(config_path_bw46)
deeplabcut.create_training_dataset(config_path_mg51b)
deeplabcut.create_training_dataset(config_path_mg117)
deeplabcut.create_training_dataset(config_path_mg118)
deeplabcut.create_training_dataset(config_path_mg120b)

In [ ]:
# Train the network for different subjects one by one (the parameters to the train_network are good I think.
# You can add something by referring the deeplabcut readme if you want to)
config_path = config_path_bw46   # Change this value for different subject training
deeplabcut.train_network(config_path,
                         shuffle=1,
                         trainingsetindex=0,
                         max_snapshots_to_keep=5,
                         autotune=False,
                         displayiters=100,
                         saveiters=20000,
                         maxiters=800000)

In [ ]:
config_path = config_path_bw46  # Change this to evaluate for different subjects
# Analyze videos (generate predictions) and plot the predictions on the videos (this takes a long time)
# This is the path to the folder where you have the test videos - the videos that were not used in either testing
# or validation.
base_vpath = '/media/data_cifs/MGH/videos/'
# Check for both avi and mp4 videos (unless they're repeated in the folder). Change according to requirement.
videos = glob.glob(os.path.join(vpath, 'BW46', '*.avi'))
videos += glob.glob(os.path.join(base_vpath, 'BW46', '*.mp4'))
deeplabcut.analyze_videos(config_path, videos, save_as_csv=True)

# This function creates the labeled video by plotting the predicted pose onto the frames so that the video can be
# visualized for judging the model's performance.
#deeplabcut.create_labeled_video(config_path, videos, save_frames=True)